# Семинар 8: RNN в задаче Language Modeling

**Language Modeling** — это задача предсказания следующего слова или символа в тексте на основе предыдущего контекста.

Примеры задач:
- Предсказать следующее слово: `"I like to drink ___"` → `"coffee"`
- Предсказать следующий символ: `"RNNs a"` → `"r"`


Зачем нужна задача Language Modeling?

1. **Автодополнение и генерация текста.**  
   Модель учится предсказывать, что будет дальше — это основа T9, autocomplete, генераторов текста и кода.

2. **Обучение языковых моделей.**  
   GPT, BERT и другие начинают обучение именно с предсказания следующего токена.

3. **Понимание структуры языка.**  
   Через предсказание следующего символа или слова модель изучает грамматику, синтаксис и типичные шаблоны языка.

Мы будем решать задачу предсказания следующего символа


![img](https://github.com/stepanovnick/dl-course/blob/main/lecture8/workshop/images/char.png?raw=true)


In [ ]:
import math
import random

# textwrap
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

#### **1.1 Генерируем текст для обучения**

In [ ]:
# Toy-example
text = """
Recurrent Neural Networks (RNNs) are a type of neural network architecture which is mainly used
to detect patterns in a sequence of data. Such data can be handwriting, genomes, text or numerical
time series which are often produced in industry settings (e.g. stock markets or sensors).
However, they are also applicable to images if these get respectively decomposed into a series of
patches and treated as a sequence
""".strip()


In [ ]:
### Сохраняем алфавит (вместо словаря)
chars = sorted(list(set(text)))
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
vocab_size = len(chars)

itos

In [ ]:
def encode(s):
  return torch.tensor([stoi[c] for c in s], dtype=torch.long)
def decode(ix):
  return "".join(itos[i] for i in ix)


In [ ]:
data = encode(text)

In [ ]:
data

In [ ]:
class CharSeqDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        ### Делим текст на отрезки
        x = self.data[idx: idx+self.seq_len]
        y = self.data[idx + 1: idx+self.seq_len + 1] ### таргет - следующий символ
        return x, y

In [ ]:
seq_len = 32
batch_size = 16

ds = CharSeqDataset(data, seq_len)
dl = DataLoader(ds, batch_size=batch_size, shuffle=True, )


In [ ]:
x, y = next(iter(dl))
x.shape, y.shape

In [ ]:
x[0]

#### **1.2 Embedings или векторизация слов**

После токенизации каждое слово или символ заменяется на целочисленный индекс. Но нейросеть не может "понять" числа как слова — ей нужны **векторы признаков**.

Слой `nn.Embedding(num_tokens, embed_dim)` создаёт **матрицу эмбеддингов**:

- Каждому индексу соответствует **вектор признаков** (например, размером 100).
- Эти векторы **обучаются вместе с моделью**.
- Похожие слова получают **похожие векторы**.

По сути, это **обучаемый словарь**: индекс → вектор


![img](https://github.com/stepanovnick/dl-course/blob/main/lecture8/workshop/images/emb.png?raw=true)

Будем использовать это дальше в модели!



#### **1.3 Vanilla RNN**

Форвард на шаге t:
$$
a_t = W_{xh} x_t + W_{hh} h_{t-1} + b_h,\quad
h_t = \tanh(a_t),\quad
o_t = W_{hy} h_t + b_y
$$

Где $x_t$ — эмбеддинг символа (используем `nn.Embedding`), $h_t$ — скрытое состояние.

Обучаем по кросс-энтропии на каждом t. Полный градиент — это обычный backprop **через развёртку по времени** (BPTT).
Чтобы стабилизировать:
- **orthogonal** инициализация для \(W_{hh}\),
- **gradient clipping** (например, 1.0).

In [ ]:
class VanillaRNNCell(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super().__init__()

        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size

        ### Инициализируем обучаемые вектора - эмбединги.
        ### Каждый вектор имеет размер embed_size. Всего vocab_size векторов.
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.Wxh = nn.Linear(embed_size, hidden_size, bias=True)
        self.Whh = nn.Linear(hidden_size, hidden_size, bias=True)
        self.Why = nn.Linear(hidden_size, output_size, bias=True)

        self.reset_parameters()

    def init_hidden(self, batch_size):
        # self.hidden_size — размер скрытого состояния
        return torch.zeros(batch_size, self.hidden_size).to(device)

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.Wxh.weight)
        nn.init.orthogonal_(self.Whh.weight)
        nn.init.xavier_uniform_(self.Why.weight)

        nn.init.zeros_(self.Wxh.bias)
        nn.init.zeros_(self.Whh.bias)
        nn.init.zeros_(self.Why.bias)

    def forward_step(self, x_t, h_prev):

        x_e = self.embed(x_t)

        a_t = self.Wxh(x_e) + self.Whh(h_prev)
        h_t = torch.tanh(a_t) # скрытое состояние

        o_t = self.Why(h_t) # логиты
        return o_t, h_t

    def forward(self, x, h0=None):
        # x: [B, T]
        B, T = x.size(0), x.size(1)
        H = self.hidden_size

        ### Инициализируем начальное состояние как 0-вектор
        h = x.new_zeros(B, H, dtype=torch.float32, device=x.device) if h0 is None else h0

        logits = []
        for t in range(T):
            o_t, h = self.forward_step(x[:, t], h) # передаем состояние h с предыдущего шага
            logits.append(o_t)

        return torch.stack(logits, dim=1), h  # [B,T,V], h_T


#### **1.4 Обучение**

Perplexity  — это метрика, которая используется для количественной оценки того, насколько хорошо вероятностная модель предсказывает выборку. Perplexity (перевод: недоумение, растерянность) является мерой **среднего числа вариантов, из которых модель должна была бы выбирать на каждом шаге**.

$$ PPL = \exp(Loss)$$

![img](https://github.com/stepanovnick/dl-course/blob/main/lecture8/workshop/images/perplexity.png?raw=true)


In [ ]:
def train_epoch(model, loader, optimizer, max_norm=1.0):

    model.train()

    loss_fn = nn.CrossEntropyLoss()
    total_loss, total_tokens = 0.0, 0

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)  # [B,T]

        optimizer.zero_grad()

        logits, *_ = model(x)              # [B,T,V]
        loss = loss_fn(logits.reshape(-1, vocab_size), y.reshape(-1))

        loss.backward()
        ### Боримся с Gradient Exploding:
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()

        total_loss += loss.item() * x.numel()   # суммируем по всем позициям
        total_tokens += x.numel()
        # print(total_tokens, x.numel())

    avg_loss = total_loss / total_tokens
    ppl = math.exp(avg_loss)
    return avg_loss, ppl

def evaluate(model, loader):
    model.eval()
    loss_fn = nn.CrossEntropyLoss()
    total_loss, total_tokens = 0.0, 0
    with torch.no_grad():
        for x,y in loader:
            # print(x.shape, y.shape)

            x = x.to(device)
            y = y.to(device)

            logits, _ = model(x)

            loss = loss_fn(logits.reshape(-1, vocab_size), y.reshape(-1))
            total_loss += loss.item() * x.numel()
            total_tokens += x.numel()
            # print(total_tokens, x.numel())
    avg_loss = total_loss / total_tokens
    ppl = math.exp(avg_loss)
    return avg_loss, ppl


In [ ]:
embed_size = 32
hidden_size = 128

rnn = VanillaRNNCell(vocab_size, embed_size, hidden_size, vocab_size).to(device)

In [ ]:
epochs = 8
lr = 3e-3

opt_rnn  = optim.Adam(rnn.parameters(),  lr=lr)

In [ ]:
print("== Train RNN ==")
for e in range(1, epochs+1):
    tr_loss, tr_ppl = train_epoch(rnn, dl, opt_rnn, max_norm=1.0)
    ev_loss, ev_ppl = evaluate(rnn, dl)
    print(f"[RNN] epoch {e:02d} | train loss {tr_loss:.4f} ppl {tr_ppl:.2f} | eval loss {ev_loss:.4f} ppl {ev_ppl:.2f}")


#### **1.5 Предсказание модели**

На этапе инференса мы подаём модели начальную строку (например, `"RNN model is"`). Модель возвращает распределение вероятностей для следующего символа. На его основе мы выбираем символ, добавляем его в строку — и подаём обратно. Так, шаг за шагом, строится текст.

Простейший вариант — **argmax**.

![img](https://github.com/stepanovnick/dl-course/blob/main/lecture8/workshop/images/words.png?raw=true)

In [ ]:
def generate_argmax(model, start_text, length=100):
    model.eval()
    x = torch.tensor([stoi[c] for c in start_text], dtype=torch.long).unsqueeze(0).to(device)

    hidden = model.init_hidden(batch_size=1)

    out = list(start_text)
    ch = x[0, -1].unsqueeze(0)  # берем последний символ

    with torch.no_grad():
        for _ in range(length):
            logits, hidden = model.forward_step(ch.unsqueeze(0), hidden) ### подаем символ ch
            ch = torch.argmax(logits, dim=-1).squeeze(0) # извлекам предсказанный символ -> сохраняем в ch
            out.append(itos[ch.item()])

    return "".join(out)


In [ ]:
print(generate_argmax(rnn, start_text="RNN model is", length=200))

In [ ]:
print(generate_argmax(rnn, start_text="a dog", length=200))

Проблемы:
- текст полностью предопределен начальным сиволом,
- может зацикливаться.

#### **1.6 Семплирование**

Вместо argmax можно выбрать следующий символ **случайно** — из распределения вероятностей, которое даёт softmax.  
Это помогает избежать зацикливания и делает текст разнообразнее.

Перед softmax можно поделить логиты на **temperature** — она управляет «остротой» распределения:

$$\text{softmax}(o_i / T) = \frac{\exp(o_i / T)}{\sum_j \exp(o_j / T)} $$

- \(T < 1\) — модель становится более уверенной,  
- \(T > 1\) — выбор становится более случайным.

![img](https://github.com/stepanovnick/dl-course/blob/main/lecture8/workshop/images/temp.png?raw=true)

Далее можно взять случайный сивол из распределения. НО полное семплирование по всему словарю может давать странные выборы (в том числе мусор). Чтобы ограничить это:

- **top-k sampling**: выбираем \( k \) самых вероятных токенов, обнуляем остальные, нормируем и семплируем из них.

- **top-p (nucleus) sampling**: выбираем минимальный набор токенов, чья суммарная вероятность ≥ \( p \) (например, 0.9).  
  Размер пула динамический, зависит от уверенности модели.


In [ ]:
def generate_sample(model, start_text, length=100, temperature=1.0, top_k=None):
    model.eval()
    x = torch.tensor([stoi[c] for c in start_text], dtype=torch.long).unsqueeze(0).to(device)

    hidden = model.init_hidden(batch_size=1)

    out = list(start_text)
    ch = x[0, -1].unsqueeze(0) # берем последний символ

    with torch.no_grad():
        for _ in range(length):
            logits, hidden = model.forward_step(ch.unsqueeze(0), hidden)

            ### Преобразование
            logits = logits.squeeze(0).squeeze(0) / temperature
            probs = F.softmax(logits, dim=-1)

            ### Выбираем топ k и нормируем
            if top_k:
                v, i = torch.topk(probs, top_k)
                probs = torch.zeros_like(probs).scatter(0, i, v)
                probs /= probs.sum()

            ### Выбираем 1 индекс с вероятностью из probs
            ch = torch.multinomial(probs, 1)
            out.append(itos[ch.item()])

    return "".join(out)


In [ ]:
generate_sample(rnn, start_text="RNN model is", length=200, temperature=0.8, top_k=10)


#### **1.7 Модель с LSTM**

In [ ]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True) # num_layers - определяет кол-во слоев LSTM стакнутых вместе
        self.fc = nn.Linear(hidden_size, output_size)

    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return (h0, c0)

    def forward(self, x, hidden=None):
        x = self.embed(x)  # [B, T, E]
        out, hidden = self.lstm(x, hidden)  # out: [B, T, H]
        logits = self.fc(out)  # [B, T, V]
        return logits, hidden

    def forward_step(self, x_t, hidden):
        x_e = self.embed(x_t)  # [B, E]
        x_e = x_e.unsqueeze(1)  # [B, 1, E] — нужно 3D на вход LSTM
        out, hidden = self.lstm(x_e, hidden)  # out: [B, 1, H]
        logits = self.fc(out.squeeze(1))  # [B, V]
        return logits, hidden



In [ ]:
# ?nn.LSTM

In [ ]:
embed_size = 32
hidden_size = 128

rnn_lstm = LSTMModel(vocab_size, embed_size, hidden_size, vocab_size).to(device)

In [ ]:
epochs = 8
lr = 3e-3

opt_rnn  = optim.Adam(rnn_lstm.parameters(),  lr=lr)

In [ ]:
print("== Train LSTM ==")
for e in range(1, epochs+1):
    tr_loss, tr_ppl = train_epoch(rnn_lstm, dl, opt_rnn, max_norm=1.0)
    ev_loss, ev_ppl = evaluate(rnn_lstm, dl)
    print(f"[LSTM] epoch {e:02d} | train loss {tr_loss:.4f} ppl {tr_ppl:.2f} | eval loss {ev_loss:.4f} ppl {ev_ppl:.2f}")
